## Import Section

In [1]:
from pprint import pprint

import datetime
import logging
import os
import random
import sys
import time


In [2]:
import numpy as np
import pandas as pd
import sklearn

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Global Variables & Major Setting Section

In [5]:
SEED = 42
random.seed(SEED)
np.random.seed(seed=SEED)

In [6]:
N = 100

### Setup Debugger & Logger

In [7]:
# FORMAT = '%(asctime)-15s %(message)s'
FORMAT = '%(message)s'
logging.basicConfig(format=FORMAT)

console = logging.StreamHandler()
console.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)

logging.getLogger('').addHandler(console)

### Input Data

In [8]:
U_nom = 690  # Volt
R_stat = 0.009 # Ohm
R_rot = 0.008  # Ohm
R_app = 0.017  # Ohm
X_Dsr = 0.068  # Ohm
s = -0.13

var_names = "U_nom,R_stat,R_rot,R_app,X_Dsr".split(',')
var_unit_measures = "Volt,Ohm,Ohm,Ohm,Ohm".split(',')
var_uim_dict = dict(zip(var_names, var_unit_measures))

result_names = "It,P_jrot,P_jstat,P_gbe,P_mecc".split(',')
result_unit_measures = "Ampere,Watt,Watt,Watt,Watt".split(',')
res_uim_dict = dict(zip(result_names, result_unit_measures))

## Functions Section

### Computation Functions

In [9]:
def compute_It_by_row(vector: np.ndarray) -> float:
    U_nom, R_stat, R_rot, R_app, s, X_Dsr = vector
    return compute_It(U_nom, R_stat, R_rot, R_app, s, X_Dsr)

def compute_It_by_X(X: np.ndarray) -> float:
    U_nom, R_stat, R_rot, R_app, s, X_Dsr = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5]

    assert all(xi < 0 for xi in s), "Error input argument 's' was greater than zero"
    
    # Compute Numerator
    numerator = U_nom / np.sqrt(3)

    # Compute Denominator
    d1 = np.power(R_stat + (R_rot+R_app)/s, 2)
    d2 = np.power(X_Dsr, 2)
    denominator = np.sqrt(d1 + d2)

    try:
        # Return result
        return numerator / denominator
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass

def compute_It(U_nom: float, R_stat: float, R_rot: float, R_app: float, s: float, X_Dsr: float) -> float:

    assert s < 0, "Error input argument 's' was greater than zero"

    # Compute Numerator
    numerator = U_nom / np.sqrt(3)

    # Compute Denominator
    d1 = np.power(R_stat + (R_rot+R_app)/s, 2)
    d2 = np.power(X_Dsr, 2)
    denominator = np.sqrt(d1 + d2)

    # pprint([numerator, denominator, d1, d2])
    logging.debug([numerator, denominator, d1, d2])
    try:
        # Return result
        return numerator / denominator
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

In [10]:
def compute_Power_by_X(X: np.ndarray) -> np.ndarray:
        
    R, It = X[:, 0], X[:, 1]

    assert all(xi < 0 for xi in s), "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 * R * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_Power_by_V(R: np.ndarray, It: np.ndarray) -> np.ndarray:
    assert s < 0, "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 * R * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_Power(R_stat: float, R_rot: float, s: float, It: float) -> float:

    try:
        # Return result
        return 3 * R * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

In [11]:
def compute_P_gap_by_X(X: np.ndarray) -> np.ndarray:
        
    R_stat, R_rot, s, It = X[:, 0], X[:, 1], X[:, 2], X[:, 3]

    assert all(xi < 0 for xi in s), "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 / s * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_P_gap_by_V(R_stat: np.ndarray, R_rot: np.ndarray, s: np.ndarray, It: np.ndarray) -> np.ndarray:
    assert s < 0, "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 / s * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_P_gap(R_stat: float, R_rot: float, s: float, It: float) -> float:
    assert s < 0, "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 / s * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

In [12]:
def compute_P_mecc_by_X(X: np.ndarray) -> np.ndarray:
        
    R_stat, R_rot, s, It = X[:, 0], X[:, 1], X[:, 2], X[:, 3]

    assert all(xi < 0 for xi in s), "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 * ((1-s) / s) * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_P_mecc_by_V(R_stat: np.ndarray, R_rot: np.ndarray, s: np.ndarray, It: np.ndarray) -> np.ndarray:
    assert s < 0, "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 * ((1-s) / s) * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

def compute_P_mecc(R_stat: float, R_rot: float, s: float, It: float) -> float:
    assert s < 0, "Error input argument 's' was greater than zero"

    try:
        # Return result
        return 3 * ((1-s) / s) * (R_rot+R_app) * It ** 2
    except Exception as err:
        # Manage Division by Zero Exception
        print(str(err), file=sys.stderr)
        sys.exit(-1)
        pass
    pass

### Create Toy Examples

In [13]:
def create_data_matrix_old(rows: int, cols: int, U_nom: float, R_stat: float, R_rot: float, R_app: float, s: float, X_Dsr: float) -> np.ndarray:
    
    assert rows > 0, "Number of rows must be strictly greater than zero"
    assert cols > 0, "Number of columns must be strictly greater than zero"

    deltas_list = [2*1e+2, 2*1e-3, 2*1e-3, 2*1e-3, -1*1e-1, 1e-3]

    U_nom_v = np.linspace(start=U_nom, stop=(U_nom+deltas_list[0]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    R_stat_v = np.linspace(start=R_stat, stop=(R_stat+deltas_list[1]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    R_rot_v = np.linspace(start=R_rot, stop=(R_rot+deltas_list[2]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    R_app_v = np.linspace(start=R_app, stop=(R_app+deltas_list[3]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    s_v = np.linspace(start=s, stop=(s+deltas_list[4]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    X_Dsr_v = np.linspace(start=X_Dsr, stop=(X_Dsr+deltas_list[5]), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)

    res = np.concatenate((U_nom_v, R_stat_v, R_rot_v, R_app_v, s_v, X_Dsr_v))
    assert len(res) == rows * cols

    X = np.zeros((rows, cols))
    for ii in range(cols):
        X[:, ii] = res[ii*rows:(ii+1)*rows]
    return X

In [14]:
def create_data_matrix_new(rows: int, cols: int, U_nom: float, R_stat: float, R_rot: float, R_app: float, s: float, X_Dsr: float, deltas_list: list = None) -> np.ndarray:
    
    assert rows > 0, "Number of rows must be strictly greater than zero"
    assert cols > 0, "Number of columns must be strictly greater than zero"
    
    if deltas_list is None:
        deltas_list = [2*1e+2, 2*1e-3, 2*1e-3, 2*1e-3, -1*1e-1, 1e-3]
    else:
        assert len(deltas_list) == cols, "deltas_list and number of columns mismatch"
        pass

    variables = [U_nom, R_stat, R_rot, R_app, s, X_Dsr]
    pairs_var_delta = list(zip(variables, deltas_list))
    
    X = np.zeros((rows, cols))
    for ii, (var, delta_var) in enumerate(pairs_var_delta):
        X[:, ii] = np.linspace(start=var, stop=(var+delta_var), num=rows, endpoint=True, retstep=False, dtype=None, axis=0)
    return X

def create_data_matrix(rows: int, cols: int, U_nom: float, R_stat: float, R_rot: float, R_app: float, s: float, X_Dsr: float, deltas_list=None, old_version: bool = False) -> np.ndarray:
    if old_version is True:
        return create_data_matrix_old(rows, cols, U_nom, R_stat, R_rot, R_app, s, X_Dsr)

    return create_data_matrix_new(rows, cols, U_nom, R_stat, R_rot, R_app, s, X_Dsr, deltas_list)

### Plot Functions

In [15]:
def plot_x_vs_y_scatter(x, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, y, color='lightblue', linewidth=3)
    ax.scatter(x, y, color='darkgreen', marker='^')
    # ax.set_xlim(0.5, 4.5)
    plt.show()
    pass

## Calculations

### Compute $I_{t}$

In [16]:
It = compute_It(U_nom, R_stat, R_rot, R_app, s, X_Dsr)
print("%.2f %s" % (It, res_uim_dict['It']))

2037.56 Ampere


In [17]:
# np.linspace(start, stop, num=N, endpoint=True, retstep=False, dtype=None, axis=0)
rows, cols = N, 6
X = create_data_matrix(rows=rows, cols=cols, U_nom=U_nom, R_stat=R_stat, R_rot=R_rot, R_app=R_app, s=s, X_Dsr=X_Dsr)

In [18]:
cols_X = "U_nom,R_stat,R_rot,R_app,s,X_Dsr".split(',')
df_X = pd.DataFrame(data=X, columns=cols_X)

df_X.head()

,U_nom,R_stat,R_rot,R_app,s,X_Dsr
0,690.000000,0.009000,0.008000,0.017000,-0.13000,0.06800
1,692.020202,0.009020,0.008020,0.017020,-0.13101,0.06801
2,694.040404,0.009040,0.008040,0.017040,-0.13202,0.06802
3,696.060606,0.009061,0.008061,0.017061,-0.13303,0.06803
4,698.080808,0.009081,0.008081,0.017081,-0.13404,0.06804


In [19]:
# np.apply_along_axis(compute_It_by_row, 1, X)
It_v = compute_It_by_X(X)

# plot_x_vs_y_scatter(X[:,0], It_v)
It_v

array([2037.56129691, 2055.25795206, 2072.9684845 , 2090.69266515,
       2108.43026763, 2126.18106822, 2143.94484584, 2161.72138201,
       2179.51046085, 2197.31186901, 2215.12539571, 2232.95083263,
       2250.78797397, 2268.63661635, 2286.49655884, 2304.36760291,
       2322.24955239, 2340.14221349, 2358.04539473, 2375.95890695,
       2393.88256323, 2411.81617896, 2429.75957172, 2447.71256131,
       2465.67496971, 2483.64662107, 2501.62734167, 2519.61695989,
       2537.61530622, 2555.62221321, 2573.63751545, 2591.66104955,
       2609.69265414, 2627.7321698 , 2645.77943908, 2663.83430648,
       2681.89661837, 2699.96622305, 2718.04297068, 2736.12671327,
       2754.21730464, 2772.31460044, 2790.4184581 , 2808.52873681,
       2826.64529752, 2844.76800291, 2862.89671733, 2881.03130687,
       2899.17163926, 2917.31758388, 2935.46901174, 2953.62579546,
       2971.78780928, 2989.95492897, 3008.12703189, 3026.30399693,
       3044.48570449, 3062.67203648, 3080.8628763 , 3099.05810

In [20]:
df_It = pd.DataFrame(data=It_v[:, np.newaxis], columns=['It'])
df_X_It = pd.concat([df_X, df_It], axis=1)

df_X_It.head()

,U_nom,R_stat,R_rot,R_app,s,X_Dsr,It
0,690.000000,0.009000,0.008000,0.017000,-0.13000,0.06800,2037.561297
1,692.020202,0.009020,0.008020,0.017020,-0.13101,0.06801,2055.257952
2,694.040404,0.009040,0.008040,0.017040,-0.13202,0.06802,2072.968484
3,696.060606,0.009061,0.008061,0.017061,-0.13303,0.06803,2090.692665
4,698.080808,0.009081,0.008081,0.017081,-0.13404,0.06804,2108.430268


### Show Results $I_{t}$

In [21]:
data = df_X_It.values
scaler = StandardScaler()
print(scaler.fit(data))

print(scaler.mean_)

data_rescaled = scaler.transform(data)
df_X_It_rescaled = pd.DataFrame(data=data_rescaled, columns=df_X_It.columns)

StandardScaler(copy=True, with_mean=True, with_std=True)
[ 7.90000000e+02  1.00000000e-02  9.00000000e-03  1.80000000e-02
 -1.80000000e-01  6.85000000e-02  2.92873437e+03]


In [22]:
# g = sns.pairplot(df_X_It_rescaled, diag_kind="kde", markers="+", \
#    plot_kws=dict(s=50, edgecolor="b", linewidth=1), \
#    diag_kws=dict(shade=True))

In [23]:
# g = sns.pairplot(df_X_It_rescaled, kind="reg")

In [24]:
# g = sns.pairplot(df_X_It_rescaled, corner=True)

### Compute $P_{mecc}$

In [25]:
target_cols = 'R_stat,R_rot,s,It'.split(',')
X_mecc = df_X_It[target_cols].values
Pmecc_v = compute_P_mecc_by_X(X=X_mecc)

In [26]:
df_Pmecc = pd.DataFrame(data=Pmecc_v[:, np.newaxis], columns=['Pmecc'])
df_It = pd.DataFrame(data=It_v[:, np.newaxis], columns=['It'])
df_results = pd.concat([df_It, df_Pmecc], axis=1)

df_results.head()

,It,Pmecc
0,2037.561297,-2.706560e+06
1,2055.257952,-2.737199e+06
2,2072.968484,-2.767972e+06
3,2090.692665,-2.798879e+06
4,2108.430268,-2.829919e+06


In [27]:
data = df_results.values
scaler = StandardScaler()
print(scaler.fit(data))

print(scaler.mean_)

data_rescaled = scaler.transform(data)
df_results_rescaled = pd.DataFrame(data=data_rescaled, columns=df_results.columns)

StandardScaler(copy=True, with_mean=True, with_std=True)
[ 2.92873437e+03 -4.42998513e+06]


### Show Results $P_{mecc}$

In [28]:
# g = sns.pairplot(df_results_rescaled, diag_kind="kde", markers="+", \
#    plot_kws=dict(s=50, edgecolor="b", linewidth=1), \
#    diag_kws=dict(shade=True))

In [29]:
 # g = sns.pairplot(df_results_rescaled, kind="reg")

In [30]:
# g = sns.pairplot(df_results_rescaled, corner=True)

### Compute $P_{gap}$

In [31]:
target_cols = 'R_stat,R_rot,s,It'.split(',')
X_gap = df_X_It[target_cols].values
P_gap_v = compute_P_gap_by_X(X=X_mecc)

In [32]:
df_P_gap_v = pd.DataFrame(data=P_gap_v[:, np.newaxis], columns=['P_gap'])
df_It = pd.DataFrame(data=It_v[:, np.newaxis], columns=['It'])

# df_results = pd.concat([df_It, df_P_gap_v], axis=1)
df_X_gap = df_X_It[target_cols]
df_results = pd.concat([df_X_gap, df_P_gap_v], axis=1)

df_results.head()

,R_stat,R_rot,s,It,P_gap
0,0.009000,0.008000,-0.13000,2037.561297,-2.395186e+06
1,0.009020,0.008020,-0.13101,2055.257952,-2.420137e+06
2,0.009040,0.008040,-0.13202,2072.968484,-2.445162e+06
3,0.009061,0.008061,-0.13303,2090.692665,-2.470260e+06
4,0.009081,0.008081,-0.13404,2108.430268,-2.495431e+06


In [33]:
data = df_results.values
scaler = StandardScaler()
print(scaler.fit(data))

print(scaler.mean_)

data_rescaled = scaler.transform(data)
df_results_rescaled = pd.DataFrame(data=data_rescaled, columns=df_results.columns)

StandardScaler(copy=True, with_mean=True, with_std=True)
[ 1.00000000e-02  9.00000000e-03 -1.80000000e-01  2.92873437e+03
 -3.73424009e+06]


### Show Results $P_{gap}$

In [34]:
# g = sns.pairplot(df_results_rescaled, diag_kind="kde", markers="+", \
#    plot_kws=dict(s=50, edgecolor="b", linewidth=1), \
#    diag_kws=dict(shade=True))

In [35]:
 # g = sns.pairplot(df_results_rescaled, kind="reg")

In [36]:
# g = sns.pairplot(df_results_rescaled, corner=True)

### Compute $P_{CEB}$, $P_{jsta}$, and $P_{jrot}$

### Show Results $P_{CEB}$, $P_{jsta}$, and $P_{jrot}$

### More Analyses

## References